In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import brown, stopwords

In [2]:
# import corpus here
# nltk.download() 

In [3]:
print(brown.categories())


['adventure', 'belles_lettres', 'editorial', 'fiction', 'government', 'hobbies', 'humor', 'learned', 'lore', 'mystery', 'news', 'religion', 'reviews', 'romance', 'science_fiction']


## I chose to only use 5 categories in my work here because I did not want overload my kernel as I am on a MacBookAir and do not have unlimited space in memory. 

In [39]:
raw_adventure = brown.sents(categories='adventure')
#raw_lore = brown.raw(categories='lore')
#raw_mystery = brown.raw(categories='mystery')
#raw_romance = brown.raw(categories='romance')
#raw_science_fiction = brown.raw(categories='science_fiction')

#raw_adventure = str(raw_adventure)
#raw_lore = str(raw_lore)
#raw_mystery = str(raw_mystery)
#raw_romance = str(raw_romance)
#raw_science_fiction = str(raw_science_fiction)


In [36]:
help(brown.raw)

Help on method raw in module nltk.corpus.reader.tagged:

raw(fileids=None, categories=None) method of nltk.corpus.reader.tagged.CategorizedTaggedCorpusReader instance
    :return: the given file(s) as a single string.
    :rtype: str



In [40]:
raw_adventure

[['Dan', 'Morgan', 'told', 'himself', 'he', 'would', 'forget', 'Ann', 'Turner', '.'], ['He', 'was', 'well', 'rid', 'of', 'her', '.'], ...]

In [48]:
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = str(text)
    text = ' '.join(text.split())
    text = re.sub('[\',]','',text)
    text = re.sub("[\[]*?![\]]", "", text)
    
    return text
    
clean_adventure = text_cleaner(raw_adventure)
#clean_lore = text_cleaner(raw_lore)
#clean_mystery = text_cleaner(raw_mystery)
#clean_romance = text_cleaner(raw_romance)
#clean_science_fiction = text_cleaner(raw_science_fiction)


In [49]:
clean_adventure[:100]

'[[Dan Morgan told himself he would forget Ann Turner .] [He was well rid of her .] ...]'

In [63]:
flat_adventure = []
for i in clean_adventure:
    flat_adventure.append(i)
        
print(flat_adventure)

['[', '[', 'D', 'a', 'n', ' ', 'M', 'o', 'r', 'g', 'a', 'n', ' ', 't', 'o', 'l', 'd', ' ', 'h', 'i', 'm', 's', 'e', 'l', 'f', ' ', 'h', 'e', ' ', 'w', 'o', 'u', 'l', 'd', ' ', 'f', 'o', 'r', 'g', 'e', 't', ' ', 'A', 'n', 'n', ' ', 'T', 'u', 'r', 'n', 'e', 'r', ' ', '.', ']', ' ', '[', 'H', 'e', ' ', 'w', 'a', 's', ' ', 'w', 'e', 'l', 'l', ' ', 'r', 'i', 'd', ' ', 'o', 'f', ' ', 'h', 'e', 'r', ' ', '.', ']', ' ', '.', '.', '.', ']']


In [51]:
nlp = spacy.load('en')

adventure_doc = nlp(clean_adventure)
#lore_doc = nlp(raw_lore)
#mystery_doc = nlp(raw_mystery)
#romance_doc = nlp(raw_romance)
#science_fiction_doc = nlp(raw_science_fiction)

In [52]:
adventure_doc

[[Dan Morgan told himself he would forget Ann Turner .] [He was well rid of her .] ...]

In [9]:
adventure_sents = [[sent, "adventure"] for sent in adventure_doc]
#lore_sents = [[sent, "lore"] for sent in lore_doc]
#mystery_sents = [[sent, "mystery"] for sent in mystery_doc]
#romance_sents = [[sent, "romance"] for sent in romance_doc]
#science_fiction_sents = [[sent, "science_fiction"] for sent in science_fiction_doc]

sentences = pd.DataFrame(adventure_sents)

sentences.head()

,0,1
0,[,adventure
1,[,adventure
2,Dan,adventure
3,,adventure
4,Morgan,adventure


In [10]:
# Utility function to create a list of the 3000 most common words.
# upped word cound from 2000 to 3000 because we aren't comparing works from
# two different authors but from 5 different genres.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(3000)]
    

def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df


In [11]:
adventure_text = brown.words(categories='adventure')
lore_text = brown.words(categories='lore')
mystery_text = brown.words(categories='mystery')
romance_text = brown.words(categories='romance')
science_fiction_text = brown.words(categories='science_fiction')

common_words = set(adventure_text + lore_text + mystery_text + romance_text + science_fiction_text)

In [12]:
word_counts = bow_features(sentences, common_words)

TypeError: 'spacy.tokens.token.Token' object is not iterable

In [ ]:
# set up pipeline pieces here
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

steps = []


In [ ]:
help(nltk.tokenize)

In [ ]:
# set up pipeline here
from sklearn.pipeline import Pipeline


In [ ]:
# put assesment here

In [ ]:
# 